In [2]:
from secrets import ShamirSecretSharing, AdditiveSecretSharing, Vandermonde, P, RandPoly

In [119]:
from sympy import *

In [120]:
import numpy as np
from numpy.linalg import inv
from scipy.interpolate import lagrange

## Description

In [121]:
# parameters
#n=7 # number of parties
#t=3 # 2t+1 <= n, subset of the parties that would reconstruct the secret of the multiplication
p=11 # suposed to be large integer value

In [122]:
# Parties
Alice = RandPoly(name="f", n=3, R=[(i,x) for i, x in enumerate(list([1,2,3]))])
Bob = RandPoly(name="g", n=3, R=[(i,x) for i, x in enumerate(list([2,3,4]))])
Charlie = RandPoly(name="h", n=3, R=[(i,x) for i, x in enumerate(list([3,4,5]))])
Alice, Bob, Charlie

(f(x)=1+2x+3x^2, g(x)=2+3x+4x^2, h(x)=3+4x+5x^2)

## Generate Shares

In [123]:
f=Alice.poly
g=Bob.poly
h=Charlie.poly
f(0), g(0), h(0)

(1, 2, 3)

In [124]:
# Shares
alice_shares = (f(1), g(1), h(1)), (f(4), g(4), h(4))
bob_shares = (f(2), g(2), h(2)), (f(5), g(5), h(5))
charlie_shares = (f(3), g(3), h(3)), (f(6), g(6),h(6))

# honest-but-curuous
server_shares = f(7), g(7), h(7)

alice_shares, bob_shares, charlie_shares, server_shares

(((6, 9, 12), (57, 78, 99)),
 ((17, 24, 31), (86, 117, 148)),
 ((34, 47, 60), (121, 164, 207)),
 (162, 219, 276))

In [125]:
# Product of shares
alice_prod = np.array([f(1)*g(1)*h(1), f(4)*g(4)*h(4)])
bob_prod = np.array([f(2)*g(2)*h(2), f(5)*g(5)*h(5)])
charlie_prod = np.array([f(3)*g(3)*h(3), f(6)*g(6)*h(6)])
server_prod = np.array(f(7)*g(7)*h(7))

alice_prod, bob_prod, charlie_prod, server_prod

(array([   648, 440154]),
 array([  12648, 1489176]),
 array([  95880, 4107708]),
 array(9791928))

## Re-randomization

In [126]:
alice_rand = RandPoly(name="q", fzc=False, n=6, R=[(i,x) for i, x in enumerate(list([0,1,3,2,5,4,1]))])
bob_rand = RandPoly(name="q", fzc=False, n=6, R=[(i,x) for i, x in enumerate(list([0,2,1,3,2,7,3]))])
charlie_rand = RandPoly(name="q", fzc=False, n=6, R=[(i,x) for i, x in enumerate(list([0,3,1,4,1,1,2]))])
alice_rand, bob_rand, charlie_rand

(q(x)=1x+3x^2+2x^3+5x^4+4x^5+1x^6,
 q(x)=2x+1x^2+3x^3+2x^4+7x^5+3x^6,
 q(x)=3x+1x^2+4x^3+1x^4+1x^5+2x^6)

In [127]:
q1 = alice_rand.poly
q2 = bob_rand.poly
q3 = charlie_rand.poly

In [128]:
alice_q = np.array([q1(1) + q2(1) + q3(1), q1(4) + q2(4) + q3(4)])
bob_q = np.array([q1(2) + q2(2) + q3(2), q1(5) + q2(5) + q3(5)])
charlie_q = np.array([q1(3) + q2(3) + q3(3), q1(6) + q2(6) + q3(6)])
server_q = np.array([q1(7) + q2(7) + q3(7)])

alice_q, bob_q, charlie_q, server_q

(array([   46, 39592]),
 array([  1000, 137530]),
 array([  8244, 385776]),
 array([930160]))

In [129]:
alice_s = alice_prod + alice_q
bob_s = bob_prod + bob_q
charlie_s = charlie_prod + charlie_q

server_s = server_prod + server_q

alice_s, bob_s, charlie_s, server_s

(array([   694, 479746]),
 array([  13648, 1626706]),
 array([ 104124, 4493484]),
 array([10722088]))

In [130]:
S = np.array([alice_s[0], bob_s[0], charlie_s[0], alice_s[1], bob_s[1], charlie_s[1]])
S = np.append(S, server_s)
# S, S.shape
S = Matrix(S.tolist())
S

Matrix([
[     694],
[   13648],
[  104124],
[  479746],
[ 1626706],
[ 4493484],
[10722088]])

## Degree Reduction

In [131]:
# Vandermonde Matrix
# B = np.array(Vandermonde(n=7))
# B = np.vander(np.array([1,2,3,4,5,6,7]), increasing=True)
B= Matrix(Vandermonde(n=7))

# Inverse Vandermonde Matrix
# Binv = inv(B)
Binv = B ** -1
# B, Binv
B, Binv

(Matrix([
 [1, 1,  1,   1,    1,     1,      1],
 [1, 2,  4,   8,   16,    32,     64],
 [1, 3,  9,  27,   81,   243,    729],
 [1, 4, 16,  64,  256,  1024,   4096],
 [1, 5, 25, 125,  625,  3125,  15625],
 [1, 6, 36, 216, 1296,  7776,  46656],
 [1, 7, 49, 343, 2401, 16807, 117649]]),
 Matrix([
 [      7,       -21,       35,      -35,      21,        -7,      1],
 [-223/20,    879/20,  -949/12,       82,  -201/4,   1019/60, -49/20],
 [ 319/45, -3929/120,    389/6, -2545/36,   134/3, -1849/120, 203/90],
 [ -37/16,      71/6, -1219/48,     88/3, -925/48,      41/6, -49/48],
 [ 59/144,      -9/4,   247/48,  -113/18,   69/16,    -19/12, 35/144],
 [  -3/80,     13/60,   -25/48,      2/3,  -23/48,     11/60, -7/240],
 [  1/720,    -1/120,     1/48,    -1/36,    1/48,    -1/120,  1/720]]))

## Linear Projection Matrix

In [132]:
# Proj = np.array(P(n=n,t=t))
# P(n=7,t=3)
Proj = Matrix(P(n=7,t=3))
Proj

Matrix([
[1, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0]])

## Matrix of product points reduction 

In [133]:
# A = B @ Proj @ Binv
# A, A.shape, S.shape
A = B * Proj * Binv
A

Matrix([
[529/180,     -235/24,     83/4,    -853/36,   185/12,  -217/40,     29/36],
[ 235/18,     -961/15,    817/6,    -1384/9,    595/6,   -104/3,    461/90],
[ 747/20,    -7353/40,   1525/4,    -1701/4,   1089/4, -3789/40,    279/20],
[3412/45,    -5536/15,      756,    -7543/9,   1604/3,   -928/5,   1228/45],
[4625/36,   -14875/24, 15125/12,  -50125/36, 10637/12, -7375/24,   1625/36],
[1953/10,        -936,   3789/2,      -2088,   2655/2,  -2299/5,     135/2],
[9947/36, -158123/120,  10633/4, -105301/36, 22295/12,  -5145/8, 16987/180]])

## Randomized polynomial outputs

In [134]:
R = A * S
R

Matrix([
[ 132],
[ 440],
[ 930],
[1602],
[2456],
[3492],
[4710]])

## Lagrange Interpolation

In [135]:
shares = [(i+1, r) for (i, r) in enumerate(R.tolist())]
R[0], shares, shares[0:3]

(132,
 [(1, [132]),
  (2, [440]),
  (3, [930]),
  (4, [1602]),
  (5, [2456]),
  (6, [3492]),
  (7, [4710])],
 [(1, [132]), (2, [440]), (3, [930])])

In [136]:
# Lagrange Interpolation for Multiplication 
x = np.arange(1,len(shares[0:3])+1) # prod points indices
y = R[0:3]      # prod points values
f = lagrange(x, y)
x, y, f

(array([1, 2, 3]),
 [132, 440, 930],
 poly1d([91.0000000000000, 35.0000000000000, 6.00000000000000], dtype=object))

## Degree reduced (and Randomized) polynomial

In [137]:
coef = [(i, r) for (i,r) in enumerate(np.flip(f.coef).tolist())]
p = RandPoly(n=t, R=coef)
p

6.00000000000000+35.0000000000000x+91.0000000000000x^2

## Secret Multiplication Result

In [138]:
p.poly(0)

6.00000000000000